# Python implementation of SHA256
Followed by the guidelines of https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

The style of the implementation is inspired by https://github.com/Eylrid/pysha256/blob/master/pysha256.py

SHA-224 and SHA-256 use the same sequence of sixty-four constant 32-bit words.  These words represent the first thirty-two bits of the fractional parts of the cube roots of the first sixty-four prime numbers. In hex, these constant words are (from left to right)

In [1]:
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2,
]

## Preprocessing
Preprocessing  consists  of  three  steps:  padding  the  message, M,  parsing  the  message into message blocks, and setting the initial hash value, H(0)

### Padding the Message
The  purpose  of  this  padding  is  to  ensure  that  the  padded  message  is  a  multiple  of  512. Padding can be inserted before hash computation begins on a message, or at any  other time during the hash computation prior to processing the block(s) that will contain the padding. 

Suppose that the length of the message M is l bits. Append the bit "1" to the end of the message, followed by k zero bits, where k is the smallest, non-negative solution to the equation $l+1+k=448mod512$. Then append the 64-bit block that is equal to the number l
expressed using a binary representation. For example, the (8-bit ASCII) message "abc" has length 8x3=24, so the message is padded with a one bit, then 448-(24+1)=423 zero bits, and then the message length, to become the 512-bit padded message
![sha256 padding](figures/sha256padding.png)

In [2]:
message = "abc"
bytearray(message, 'ascii')

bytearray(b'abc')

In [3]:
bitlen = len(message)*8
bitlen

24

In [4]:
def pad_len(bitlen):
    """Calculates number of bits needed for message padding
    Args:
        bitlen: number of bits in the message
        
    Returns:
        bitstring to pad after the message. "1" followed by
        k zero bits where l+1+k=448mod512. l=message length.
    """
    length = 448-(bitlen%512)
    if length < 0:
        length += 512
    return '1'.ljust(length,'0')
    
padding = pad_len(bitlen)
print(f'Pad the message: {message} with "1" then {len(padding)-1} zero bits')
padding

Pad the message: abc with "1" then 423 zero bits


'1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [5]:
def block_bits(bitlen):
    """Calculates 64-bit block to be added at the end of message
    Args:
        bitlen: number of bits in the original message (before padding)
        
    Returns:
        (String) 64-bit block which is a binary representation of the
        message length
    """
    bits = bin(bitlen)[2:].rjust(64,'0')
    assert len(bits) == 64
    return bits

len_bits = block_bits(bitlen)
print(f'Add 64-bit block in the end:\n{len_bits}')

Add 64-bit block in the end:
0000000000000000000000000000000000000000000000000000000000011000


In [6]:
def char_to_bitstring(char):
    """Converts an ASCII character to 8-bit string"""
    return bin(ord(char))[2:].rjust(8,"0")

print(f'a: {char_to_bitstring("a")}')
print(f'b: {char_to_bitstring("b")}')
print(f'c: {char_to_bitstring("c")}')

a: 01100001
b: 01100010
c: 01100011


In [7]:
def pad_message(message):
    """Adds padding to an ASCII message
    Args:
        message: original message in ASCII characters
        
    Returns:
        bitstring which is a multiple of 512. The bitstring
        is padded based SHA-256 algorithm.
    
    """
    bitstring = ''.join([char_to_bitstring(c) for c in message])
    bitlen = len(bitstring)
    padding = pad_len(bitlen)
    len_bits = block_bits(bitlen)
    # Add padding and 64-bit message block
    bitstring = bitstring + padding + len_bits
    assert len(bitstring)%512==0
    return bitstring
    
bitstring = pad_message(message)
bitstring

'01100001011000100110001110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000'

### Parsing the message
The message and its padding must be parsed into N m-bit blocks

For SHA-256, the message and its padding are parsed into N 512-bit blocks, $M^{(1)},M^{(2)},..,M^{(N)}$. Since the 512 bits of the input block may be expressed as sixteen 32-bit words, the first 32 bits of message block i are denoted $M_{0}^{(i)}$, the next 32 bits are $M_{1}^{(i)}$, and so on up to $M_{15}^{(i)}$

In [8]:
def parse_block(bitstring):
    """Parse message into 16 32-bit blocks
    Args:
        bitstring: bitstring of length 512
    
    Returns:
        list of 16 32-bit bitstrings
    """
    M = [bitstring[i:i + 32] for i in range(0, 512, 32)]
    assert len(M) == 16
    return M

M = parse_block(bitstring)
M

['01100001011000100110001110000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000000000',
 '00000000000000000000000000011000']

### Setting initial Hash Value 
Before hash computation begins, the initial hash value, $H^{(0)}$ must be set. The size and number of words in $H^{(0)}$ depends on the message digest size

For SHA-256, the initial hash value $H^{(0)}$ shall consist of the following eight 32-bit words in hex:
* $H_{0}^{(0)}$ = 6a09e667
* $H_{1}^{(0)}$ = bb67ae85
* $H_{2}^{(0)}$ = 3c6ef372
* $H_{3}^{(0)}$ = a54ff53a
* $H_{4}^{(0)}$ = 510e527f
* $H_{5}^{(0)}$ = 9b05688c
* $H_{6}^{(0)}$ = 1f83d9ab
* $H_{7}^{(0)}$ = 5be0cd19

These words were obtained by taking the first thirty-two bits of the fractional parts of the square roots of the first eight prime numbers

In [9]:
h0 = 0x6a09e667
h1 = 0xbb67ae85
h2 = 0x3c6ef372
h3 = 0xa54ff53a
h4 = 0x510e527f
h5 = 0x9b05688c
h6 = 0x1f83d9ab
h7 = 0x5be0cd19

### SHA-256 algorithm
SHA-256 may be used to hash a message, M, having a length of l bits, where 0<=l<=$2^{64}$. The algorithm uses

1. A message schedule of sixty four 32-bit words
2. Eight working variables of 32-bit each
3. A hash value of eight 32-bit words

The final value of SHA-256 is a 256-bit message digest.

The words of the message schedule are labeled $W_{0}, W_{1},..,W_{63}$. The eight working variables are labeled a, b, c, d, e, f, g and h. The words of the hash value are labeled $H_{0}^{(i)}, H_{1}^{(i)},.., H_{7}^{(i)}$, which will hold the initial hash value, $H^{(0)}$, replaced by each successive intermediate hash value (after each message block is processed), $H^{(i)}$, and ending with the final hash value, $H^{(N)}$. SHA-256 also uses two temporary words, $T_{1}$ and $T_{2}$.

#### Preprocessing
1. Set initial hash value $H^{(0)}$
2. Pad message (as above)

#### Hash Computation
The SHA-256 hash computation uses a set functions and constants. Additions (+) is performed modulo $2^{32}$

Each message block, $M^{1}, M^{2},.., M^{N}$, is processed in order, using the following steps:
![sha256 padding](figures/sha256algo1.png)
![sha256 padding](figures/sha256algo2.png)

### Handling bitstrings
We will create a class that handles bitstrings. A bitstring will in this case be a list of integer bits (0/1) that we are able to perform all functions needed for SHA-256 on.

In [194]:
class BitString:
    """Helper class for creation and manupilation of binary data"""
    
    def __init__(self, bits):
        """Initiate a BitString object
        Args:
            bits: list of integer (0/1) values 
        """
        self.bits = bits
        self._validate_bits()
        
    @classmethod
    def from_hex(cls, hexstring, n):
        """Initiate a BitString object from hex values
        Args:
            hexstring: integer in hex format
            n: number of bits in BitString
        """
        bits = bin(hexstring)[2:].rjust(n,'0')
        bits = [int(i) for i in bits]
        return cls(bits)
    
    @classmethod
    def from_bitstring(cls, bitstring):
        """Initiate a BitString object from string of bits
        Args:
            bitstring: string of bits (0/1)
        """
        bits = [int(i) for i in bitstring]
        return cls(bits)
    
    @staticmethod
    def to_bitstring(n, l):
        """Converts an integer (n) to a (l) length bitstring"""
        return bin(n)[2:].rjust(l,'0')
    
    def copy(self):
        return BitString(self.bits[:])
    
    def _validate_bits(self):
        """Ensure all bits are either 0 or 1"""
        if any([bit not in [0,1] for bit in set(self.bits)]):
            raise ValueError('Bits should be 0 or 1')
        return
    
    def to_int(self):
        """Returns integer representation of self"""
        bitstring = ''.join(str(i) for i in self.bits)
        return int(bitstring, 2)
    
    def to_string(self):
        """Returns ASCII string representation of self"""
        assert len(self.bits)%8 == 0
        string = []
        for i in range(0, len(self.bits), 8):
            byte = self.bits[i:i+8]
            byte = ''.join([str(i) for i in byte])
            string.append(chr(int(byte,2)))
        return ''.join(string)
    
    def to_hex(self):
        """Returns hex representation of self"""
        return self.to_string().encode().hex()
    
    def add(self, bits):
        """Performs addition in modulo 2^32
        Args:
            bits (BitString): object with bits to be added with self
        """
        if not isinstance(bits, BitString):
            raise TypeError('Addition is only allowed with another BitString Object')
        num = (self.to_int() + bits.to_int())%pow(2,32)
        bitstring = self.to_bitstring(num, len(self.bits))
        self.bits = [int(i) for i in bitstring]
        self._validate_bits()
        #sn = self.to_int()
        #on = bits.to_int()
        #n = (sn + on)%(2**len(self.bits))
        #bitstring = self.to_bitstring(n, len(self.bits))
        #return int_to_bitstring(n, len(self))
        return BitString(self.bits)
        
    def extend(self, bits):
        """Appends new bits to itself
        Args:
            bits (BitString): object with bits to be appended to itself
        """
        if not isinstance(bits, BitString):
            raise TypeError('Extention is only allowed with another BitString Object')
        self.bits.extend(bits.bits)
        return BitString(self.bits[:])
        
    def unshift(self, bits):
        """Adds new bits to the beginning of self
        Args: 
            bits (BitString): object with bits to be added to itself
        """
        if not isinstance(bits, BitString):
            raise TypeError('Extention is only allowed with another BitString Object')
        self.bits = bits.bits + self.bits
        return BitString(self.bits[:])
        
    def rotate_right(self, n):
        """Performs a circular right shift operation
        Takes n last bits (right) and places them in front (left)
        
        Args:
            n: number to be shifted
        """
        self.bits = self.bits[-n:] + self.bits[:-n]
        return BitString(self.bits[:])
            
            
    def shift_right(self, n):
        """Shifts (self) bits n places to the right
        Remaining bits to the left becomes zero
        """
        left_pad = [0]*n
        self.bits = left_pad + self.bits[:-n]
        return BitString(self.bits[:])
        
    def bitwise_xor(self, bits):
        """Performs a bitwise XOR with another BitString
        Args:
            bits (BitString): object with bits to perform XOR with
        """
        assert len(self.bits) == len(bits.bits)
        if not isinstance(bits, BitString):
            raise TypeError('XOR is only allowed with another BitString Object')
        
        self.bits = [0 if i==j else 1 for i,j in zip(self.bits, bits.bits)]
        return BitString(self.bits[:])
        
    def bitwise_and(self, bits):
        """Performs a bitwise AND with another BitString
        Args:
            bits (BitString): object with bits to perform AND with
        """
        if not isinstance(bits, BitString):
            raise TypeError('Bitwise AND is only allowed with another BitString Object')
        self.bits = [1 if i and j else 0 for i,j in zip(self.bits, bits.bits)]
        return BitString(self.bits[:])
        
    def bitwise_not(self):
        """Performs a bitwise NOT with another BitString"""
        self.bits = [1 if not bit else 0 for bit in self.bits]
        return BitString(self.bits[:])
        
        

In [190]:
b = BitString.from_bitstring('10111101')
print(f'b: {b.bits}')
b.shift_right(3)
b.bits

b: [1, 0, 1, 1, 1, 1, 0, 1]


[0, 0, 0, 1, 0, 1, 1, 1]

In [191]:
a = BitString.from_hex(h0, 32)
b = BitString.from_bitstring('10111101')
print(f'a: {a.to_int()}\nb: {b.to_int()}')
print(b.bits)
print(a.bits)
a.unshift(b)
print(a.bits)
a.extend(b)

a: 1779033703
b: 189
[1, 0, 1, 1, 1, 1, 0, 1]
[0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1]
[1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1]


In [195]:
a = BitString.from_bitstring('10111101')
b = BitString.from_bitstring('10111101')
print(f'a: {a.to_int()}\nb: {b.to_int()}')
print(a.bits)
print(b.bits)
a.add(b)
print(a.bits)
print(a.to_int())

a: 189
b: 189
[1, 0, 1, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 1, 0, 1]
[1, 0, 1, 1, 1, 1, 0, 1, 0]
378


In [33]:
a.rotate_right(5).rotate_right(5)

### Process the bitstring
Convert the preprocessed message into a BitString and run the hashing algorithm

In [122]:
h0 = 0x6a09e667
h1 = 0xbb67ae85
h2 = 0x3c6ef372
h3 = 0xa54ff53a
h4 = 0x510e527f
h5 = 0x9b05688c
h6 = 0x1f83d9ab
h7 = 0x5be0cd19
H = [h0, h1, h2, h3, h4, h5, h6, h7]

In [196]:
def sha256(message, H=H, K=K):
    bitstring = pad_message(message)
    chunks = [] # bitstring chunks of 512 bits
    for i in range(0, len(bitstring), 512):
        chunks.append(bitstring[i:i+512])
    # Initialize hash values as BitString objects
    # They are originally 32 bit hex values
    h0,h1,h2,h3,h4,h5,h6,h7 = list(map(lambda x: BitString.from_hex(x, 32), H))
    # Initialize round constants as BitString objects
    K = [BitString.from_hex(k, 32) for k in K]
    for chunk in chunks: # chunk is bitstring of size 512
        # Run 64 rounds in compression function
        # We need W(i) and K(i) for i=1,..,64
        # Break the 16 first rounds into 32-bit blocks from the chunk
        M = parse_block(chunk) 
        # The 48 next rounds will be computed by
        # W(i) = Wⁱ⁻¹⁶ + σ⁰ + Wⁱ⁻⁷ + σ¹ where,
        # σ⁰ = (Wⁱ⁻¹⁵ ROTR⁷(x)) XOR (Wⁱ⁻¹⁵ ROTR¹⁸(x)) XOR (Wⁱ⁻¹⁵ SHR³(x))
        # σ¹ = (Wⁱ⁻² ROTR¹⁷(x)) XOR (Wⁱ⁻² ROTR¹⁹(x)) XOR (Wⁱ⁻² SHR¹⁰(x))
        # ROTRⁿ(x) = Circular right rotation of 'x' by 'n' bits
        # SHRⁿ(x)  = Circular right shift of 'x' by 'n' bits
        W = M + ['0'.rjust(32,'0')]*48
        W = [BitString.from_bitstring(w) for w in W]
        for i in range(16, 64):
            _W = [w.copy() for w in W]
            sigma1 = _W[i-15].rotate_right(7).bitwise_xor(_W[i-15].rotate_right(18))\
            .bitwise_xor(_W[i-15].shift_right(3))
            sigma2 = _W[i-2].rotate_right(17).bitwise_xor(_W[i-2].rotate_right(19))\
            .bitwise_xor(_W[i-2].shift_right(10))
            W[i] = _W[i-16].add(sigma1).add(_W[i-7]).add(sigma1)
        # Initialize eight working variables
        a,b,c,d,e,f,g,h = h0,h1,h2,h3,h4,h5,h6,h7
        # Compression function
        
        for i in range(64):
            _e = e.copy()
            _a = a.copy()
            _c = c.copy()
            _f = f.copy()
            _h = h.copy()
            _g = g.copy()
            _b = b.copy()
            sum_e = _e.rotate_right(6).bitwise_xor(_e.rotate_right(11))\
            .bitwise_xor(_e.rotate_right(25))
            sum_a = _a.rotate_right(2).bitwise_xor(_a.rotate_right(13))\
            .bitwise_xor(_a.rotate_right(22))
            Ch = _e.bitwise_and(_f).bitwise_xor(_e.bitwise_not().bitwise_and(_g))
            maj = _a.bitwise_and(_b).bitwise_xor(_a.bitwise_and(_c))\
            .bitwise_xor(_b.bitwise_and(_c))
            T1 =  _h.add(sum_e).add(Ch).add(K[i]).add(W[i])
            T2 = sum_e.add(maj)
            h,g,f,e,d,c,b,a = g,f,e,d.add(T1),c,b,a,T1.add(T2)
        
        # Add compressed chunk to current hash val
        for h, char in zip([h0,h1,h2,h3,h4,h5,h6,h7],[a,b,c,d,e,f,g,h]):
            char = BitString.from_bitstring(char.bits)
        # Produce final hash val
        digest = h0.extend(h1).extend(h2).extend(h3).extend(h4)\
                .extend(h5).extend(h6).extend(h7)
        return digest

In [200]:
digest = sha256('abc')
digest.to_hex()

'c3903e4019c3b2c3b74c4bc2b80ec293c3a9c290550174510e527fc29b0568c28c1fc283c399c2ab5bc3a0c38d19'

In [148]:
import hashlib

In [173]:
true = hashlib.sha256('abc'.encode()).hexdigest()
true

'ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad'

In [174]:
len(true)

64